In [16]:
import numpy as np
from paddlenlp.metrics import Distinct

In [67]:
with open("../src_v7_sent_prmpt_ele-wise-mul/res/new_lr1e-5_ep5_add_bs96_kl02_tk1_tp0_tm1o1_rpt1o5_wo_topic.txt", "r") as f:
    data1 = f.readlines()
data1 = [item.strip() for item in data1]
with open("../src_v7_sent_prmpt_ele-wise-mul/res/new_lr1e-5_ep5_add_bs96_kl02_tk1_tp0_tm1o1_rpt1o5_wo_topic_disorder.txt", "r") as f:
    data2 = f.readlines()
data2 = [item.strip() for item in data2]

with open("../src_v5_refine/res/lr1e-5_bs96_kl02_tk1_tp0_tm1o1_rpt1o5_wo_topic.txt", "r") as f:
    data3 = f.readlines()
data3 = [item.strip() for item in data3]
with open("../src_v5_refine/res/lr1e-5_bs96_kl02_tk1_tp0_tm1o1_rpt1o5_wo_topic_disorder.txt", "r") as f:
    data4 = f.readlines()
data4 = [item.strip() for item in data4]

In [50]:
len(data1), data1[0], len(data2), data2[0]

(50,
 '我们的爱情，是一个美丽的传说，你和她在那里，有着甜蜜又温馨，这样幸福，就像天，让人心跳，每次呼吸，都会感觉到，好奇妙',
 500,
 '我们的爱情，，是不能永远，，你说过要和他在一起，，可现实却让人心碎，这样下去，，，还有什么意义，如果真的没有结局，，就别再为了谁，把自己伤得那麽深，，也许只剩下回忆，才会更加明白，珍，我想对你好，但又怕失败')

## 计算delta_distinct (disorder的distinct求平均 - order的distinct) => 不理想

In [5]:
def cal_delta_dist(data1, data2, n):
    res = []
    for i in range(len(data1)):
        ref = data1[i]
        lines = data2[i*10:(i+1)*10]
        distinct1 = Distinct(n_size=n)
        distinct1.add_inst(list(ref))
        tmp = []
        for line in lines:
            distinct2 = Distinct(n_size=n)
            distinct2.add_inst(list(line))
            tmp.append(distinct2.score())
        res.append(np.mean(tmp) - distinct1.score())
    print(np.mean(res))

In [7]:
n = 2
cal_delta_dist(data1, data2, n)
cal_delta_dist(data3, data4, n)

0.016364210533163743
0.00034822074323330064


## 计算cal_new_ngrams_rate => 还可以

In [59]:
def create_ngram_set(input_list, ngram_num):
    if len(input_list) < ngram_num:
        return set()
    if len(input_list) <= ngram_num:
        return {tuple(list(input_list))}
    else:
        return set(zip(*[input_list[i:] for i in range(ngram_num)]))

In [60]:
a = create_ngram_set("这是测试文本，测试的文本",2)
b = create_ngram_set("另一个文本", 2)
print(a, b, a - b)

{('这', '是'), ('试', '的'), ('是', '测'), ('试', '文'), ('的', '文'), ('文', '本'), ('本', '，'), ('测', '试'), ('，', '测')} {('个', '文'), ('文', '本'), ('另', '一'), ('一', '个')} {('这', '是'), ('试', '的'), ('是', '测'), ('的', '文'), ('试', '文'), ('本', '，'), ('测', '试'), ('，', '测')}


In [61]:
def cal_new_ngrams_rate(ref_data, thep_data, n):
    res = []
    for i in range(len(ref_data)):
        ref = ref_data[i]
#         lines = [thep_data[i]]
        lines = thep_data[i*10:(i+1)*10]
        ref_unique_ngrams = create_ngram_set(ref, n)
        tmp = []
        for line in lines:
            tmp_unique_ngrams = create_ngram_set(line, n)
            tmp.append(len(tmp_unique_ngrams-ref_unique_ngrams)/len(ref_unique_ngrams|tmp_unique_ngrams))
        res.append(np.mean(tmp))
    print(np.mean(res))

In [69]:
n = 2
cal_new_ngrams_rate(data1, data2, n)
cal_new_ngrams_rate(data3, data4, n)

0.3706678720105774
0.4053859127373593


## 计算uni(AUB)/AUB => 不理想

In [25]:
def create_ngram_set(input_list, ngram_num):
    if len(input_list) < ngram_num:
        return set()
    if len(input_list) <= ngram_num:
        return {tuple(list(input_list))}
    else:
        return list(zip(*[input_list[i:] for i in range(ngram_num)]))

In [26]:
a = create_ngram_set("这是测试文本",2)
b = create_ngram_set("另一个文本", 2)
print(a, b)

[('这', '是'), ('是', '测'), ('测', '试'), ('试', '文'), ('文', '本')] [('另', '一'), ('一', '个'), ('个', '文'), ('文', '本')]


In [27]:
def cal_uni_ngrams_rate(ref_data, thep_data, n):
    res = []
    for i in range(len(ref_data)):
        ref = ref_data[i]
        lines = thep_data[i*10:(i+1)*10]
        ref_sents = ref.split('，')
        ref_unique_ngrams = []
        for sent in ref_sents:
            ref_unique_ngrams += create_ngram_set(sent, n)
        tmp = []
        for line in lines:
            line_sents = line.split('，')
            tmp_unique_ngrams = []
            for sent in line_sents:
                tmp_unique_ngrams += create_ngram_set(sent, n)
#             tmp.append(len(set(tmp_unique_ngrams)-set(ref_unique_ngrams))/len(set(ref_unique_ngrams)|set(tmp_unique_ngrams)))
            tmp.append(len(set(tmp_unique_ngrams+ref_unique_ngrams))/(len(ref_unique_ngrams)+len(tmp_unique_ngrams)))
#             tmp.append(len(set(tmp_unique_ngrams)-set(ref_unique_ngrams))/(len(set(ref_unique_ngrams))+len(set(tmp_unique_ngrams))))
        res.append(np.mean(tmp))
    print(np.mean(res))

In [29]:
n = 2
cal_uni_ngrams_rate(data1, data2, n)
cal_uni_ngrams_rate(data3, data4, n)

0.8578730258117452
0.8338766599619114
